<h1>Clean CSV Data and Add to SQL Database</h1>

<h4>Import Dependencies</h4>

In [1]:
import os
import csv
import pandas as pd
import random

import sqlite3
import csv
from sqlalchemy import create_engine

from config import pgPassword

<h4>Create paths to CSV files</h4>

In [2]:
pathTeams = os.path.join("..", "..", "data", "csv", "Teams.csv")
pathBatting = os.path.join("..", "..", "data", "csv", "Batting.csv")
pathPitching = os.path.join("..", "..", "data", "csv", "Pitching.csv")
pathPlayers = os.path.join("..", "..", "data", "csv", "People.csv")
pathFranchises = os.path.join("..", "..", "data", "csv", "TeamsFranchises.csv")
pathSalaries = os.path.join("..", "..", "data", "csv", "Salaries.csv")

<h4>Load CSV files into DataFrame and clean</h4>

In [3]:
# Read csv into a DataFrame
franchisesDF = pd.read_csv(pathFranchises)
# Trim off unwanted columns
franchisesCols = ["franchID", "franchName"]
franchisesDF = franchisesDF[franchisesCols]
franchisesDF = franchisesDF.rename(columns={'franchID': 'franchiseID'})
franchisesDF = franchisesDF.rename(columns={'franchName': 'FranchiseName'})
franchisesDF

,franchiseID,FranchiseName
0,ALT,Altoona Mountain City
1,ANA,Los Angeles Angels of Anaheim
2,ARI,Arizona Diamondbacks
3,ATH,Philadelphia Athletics
4,ATL,Atlanta Braves
...,...,...
115,WNL,Washington Nationals
116,WNT,Washington Nationals
117,WOR,Worcester Ruby Legs
118,WSN,Washington Nationals


In [4]:
# Read csv into a DataFrame
teamsDF = pd.read_csv(pathTeams)
# Trim off unwanted columns
teamsCols = ["franchID", "teamID", "name", "yearID", "G", "W", "R", "H", "HR", "BB", "SO", "SB", "RA", "ERA", "HA", "HRA", "BBA", "SOA"]
teamsDF = teamsDF[teamsCols]
teamsDF = teamsDF.rename(columns={'franchID': 'franchiseID'})
teamsDF['statID'] = teamsDF.index
teamsDF['teamIDy'] = teamsDF.index
for i in range(len(teamsDF)): 
     teamsDF.loc[i, 'statID'] = ((i+1)**2)+999,
     teamsDF.loc[i, 'teamIDy'] = teamsDF.loc[i,'teamID'] + "-" + str(teamsDF.loc[i,'yearID'])

teamsDF.head()

,franchiseID,teamID,name,yearID,G,W,R,H,HR,BB,SO,SB,RA,ERA,HA,HRA,BBA,SOA,statID,teamIDy
0,BNA,BS1,Boston Red Stockings,1871,31,20,401,426,3,60.0,19.0,73.0,303,3.55,367,2,42,23,1000,BS1-1871
1,CNA,CH1,Chicago White Stockings,1871,28,19,302,323,10,60.0,22.0,69.0,241,2.76,308,6,28,22,1003,CH1-1871
2,CFC,CL1,Cleveland Forest Citys,1871,29,10,249,328,7,26.0,25.0,18.0,341,4.11,346,13,53,34,1008,CL1-1871
3,KEK,FW1,Fort Wayne Kekiongas,1871,19,7,137,178,2,33.0,9.0,16.0,243,5.17,261,5,21,17,1015,FW1-1871
4,NNA,NY2,New York Mutuals,1871,33,16,302,403,1,33.0,15.0,46.0,313,3.72,373,7,42,22,1024,NY2-1871


In [5]:
teamsDF[pd.isnull(teamsDF).any(axis=1)]

,franchiseID,teamID,name,yearID,G,W,R,H,HR,BB,SO,SB,RA,ERA,HA,HRA,BBA,SOA,statID,teamIDy
18,OLY,WS3,Washington Olympics,1872,9,2,54,91,0,4.0,4.0,NaN,140,6.38,148,0,5,1,1360,WS3-1872
19,NAT,WS4,Washington Nationals,1872,11,0,80,99,0,1.0,3.0,NaN,190,6.18,193,2,3,2,1399,WS4-1872
21,MAR,BL4,Baltimore Marylands,1873,6,0,26,33,0,NaN,3.0,NaN,152,8.00,144,1,1,0,1483,BL4-1873
50,ATL,BSN,Boston Red Caps,1876,70,39,471,723,9,58.0,98.0,NaN,450,2.51,732,7,104,77,3600,BSN-1876
51,CHC,CHN,Chicago White Stockings,1876,66,52,624,926,8,70.0,45.0,NaN,257,1.76,608,6,29,51,3703,CHN-1876
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
566,DET,DET,Detroit Tigers,1912,154,69,720,1376,19,530.0,NaN,270.0,777,3.77,1438,16,521,512,322488,DET-1912
568,NYY,NYA,New York Highlanders,1912,153,50,630,1320,18,463.0,NaN,247.0,842,4.13,1448,28,436,637,324760,NYA-1912
569,OAK,PHA,Philadelphia Athletics,1912,153,90,779,1442,22,485.0,NaN,258.0,658,3.32,1273,12,518,601,325899,PHA-1912
572,BAL,SLA,St. Louis Browns,1912,157,53,552,1262,19,449.0,NaN,176.0,764,3.71,1433,17,442,547,329328,SLA-1912


In [6]:
teamsDF2 = pd.DataFrame()
teamsDF2['teamID'] = teamsDF['teamIDy']
teamsDF2['TeamName'] = teamsDF['name']
teamsDF2 = teamsDF2.drop_duplicates()
teamsDF2

,teamID,TeamName
0,BS1-1871,Boston Red Stockings
1,CH1-1871,Chicago White Stockings
2,CL1-1871,Cleveland Forest Citys
3,FW1-1871,Fort Wayne Kekiongas
4,NY2-1871,New York Mutuals
...,...,...
2920,SLN-2019,St. Louis Cardinals
2921,TBA-2019,Tampa Bay Rays
2922,TEX-2019,Texas Rangers
2923,TOR-2019,Toronto Blue Jays


In [7]:
team_statsDF = teamsDF.copy()
team_statsDF = team_statsDF[team_statsDF.columns.drop('name')]
team_statsDF = team_statsDF[team_statsDF.columns.drop('teamID')]
team_statsDF = team_statsDF.rename(columns={'teamIDy': 'teamID'})
team_statsDF

,franchiseID,yearID,G,W,R,H,HR,BB,SO,SB,RA,ERA,HA,HRA,BBA,SOA,statID,teamID
0,BNA,1871,31,20,401,426,3,60.0,19.0,73.0,303,3.55,367,2,42,23,1000,BS1-1871
1,CNA,1871,28,19,302,323,10,60.0,22.0,69.0,241,2.76,308,6,28,22,1003,CH1-1871
2,CFC,1871,29,10,249,328,7,26.0,25.0,18.0,341,4.11,346,13,53,34,1008,CL1-1871
3,KEK,1871,19,7,137,178,2,33.0,9.0,16.0,243,5.17,261,5,21,17,1015,FW1-1871
4,NNA,1871,33,16,302,403,1,33.0,15.0,46.0,313,3.72,373,7,42,22,1024,NY2-1871
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2920,STL,2019,162,91,764,1336,210,561.0,1420.0,116.0,662,3.80,1284,191,545,1399,8533240,SLN-2019
2921,TBD,2019,162,96,769,1427,217,542.0,1493.0,94.0,656,3.65,1274,181,453,1621,8539083,TBA-2019
2922,TEX,2019,162,78,810,1374,223,534.0,1578.0,131.0,878,5.06,1515,241,583,1379,8544928,TEX-2019
2923,TOR,2019,162,67,726,1299,247,509.0,1514.0,51.0,828,4.79,1450,228,604,1332,8550775,TOR-2019


In [8]:
team_statsDF2 = team_statsDF.fillna(0)
team_statsDF2[pd.isnull(team_statsDF2).any(axis=1)]

,franchiseID,yearID,G,W,R,H,HR,BB,SO,SB,RA,ERA,HA,HRA,BBA,SOA,statID,teamID


In [9]:
team_statsDF2

,franchiseID,yearID,G,W,R,H,HR,BB,SO,SB,RA,ERA,HA,HRA,BBA,SOA,statID,teamID
0,BNA,1871,31,20,401,426,3,60.0,19.0,73.0,303,3.55,367,2,42,23,1000,BS1-1871
1,CNA,1871,28,19,302,323,10,60.0,22.0,69.0,241,2.76,308,6,28,22,1003,CH1-1871
2,CFC,1871,29,10,249,328,7,26.0,25.0,18.0,341,4.11,346,13,53,34,1008,CL1-1871
3,KEK,1871,19,7,137,178,2,33.0,9.0,16.0,243,5.17,261,5,21,17,1015,FW1-1871
4,NNA,1871,33,16,302,403,1,33.0,15.0,46.0,313,3.72,373,7,42,22,1024,NY2-1871
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2920,STL,2019,162,91,764,1336,210,561.0,1420.0,116.0,662,3.80,1284,191,545,1399,8533240,SLN-2019
2921,TBD,2019,162,96,769,1427,217,542.0,1493.0,94.0,656,3.65,1274,181,453,1621,8539083,TBA-2019
2922,TEX,2019,162,78,810,1374,223,534.0,1578.0,131.0,878,5.06,1515,241,583,1379,8544928,TEX-2019
2923,TOR,2019,162,67,726,1299,247,509.0,1514.0,51.0,828,4.79,1450,228,604,1332,8550775,TOR-2019


In [10]:
# Read csv into a DataFrame
playersDF = pd.read_csv(pathPlayers)
# Trim off unwanted columns
playersCols = ["playerID", "birthYear", "nameFirst", "nameLast", "debut", "finalGame"]
playersDF = playersDF[playersCols]
playersDF.head()

,playerID,birthYear,nameFirst,nameLast,debut,finalGame
0,aardsda01,1981.0,David,Aardsma,2004-04-06,2015-08-23
1,aaronha01,1934.0,Hank,Aaron,1954-04-13,1976-10-03
2,aaronto01,1939.0,Tommie,Aaron,1962-04-10,1971-09-26
3,aasedo01,1954.0,Don,Aase,1977-07-26,1990-10-03
4,abadan01,1972.0,Andy,Abad,2001-09-10,2006-04-13


In [11]:
playersDF[pd.isnull(playersDF).any(axis=1)]

,playerID,birthYear,nameFirst,nameLast,debut,finalGame
57,actama99,1969.0,Manny,Acta,NaN,NaN
59,adairbi99,1913.0,Bill,Adair,NaN,NaN
490,armoubi99,1869.0,Bill,Armour,NaN,NaN
762,bancrfr99,1846.0,Frank,Bancroft,NaN,NaN
832,barlial99,1915.0,Al,Barlick,NaN,NaN
...,...,...,...,...,...,...
19893,yawketo99,1903.0,Tom,Yawkey,NaN,NaN
19930,yostgu01,NaN,Gus,Yost,1893-06-12,1893-06-12
19976,youngni99,1840.0,Nick,Young,NaN,NaN
20026,zay01,NaN,William,Zay,1886-10-07,1886-10-07


In [12]:
playersDF2 = playersDF.copy()
playersDF2['birthYear'] = playersDF2['birthYear'].fillna(0)
playersDF2['debut'] = playersDF2['debut'].fillna('2021-01-01')
playersDF2['finalGame'] = playersDF2['finalGame'].fillna('2021-01-01')
playersDF2['nameFirst'] = playersDF2['nameFirst'].fillna("NA")
playersDF2[pd.isnull(playersDF2).any(axis=1)]

,playerID,birthYear,nameFirst,nameLast,debut,finalGame


In [13]:
# Read csv into a DataFrame
salariesDF = pd.read_csv(pathSalaries)
# Trim off unwanted columns
salariesCols = ["yearID", "teamID", "playerID", "salary"]
salariesDF = salariesDF[salariesCols]
salariesDF.head()

,yearID,teamID,playerID,salary
0,1985,ATL,barkele01,870000
1,1985,ATL,bedrost01,550000
2,1985,ATL,benedbr01,545000
3,1985,ATL,campri01,633333
4,1985,ATL,ceronri01,625000


In [14]:
FP_df = salariesDF.copy()
FP_df['fpID'] = FP_df.index
for i in range(len(FP_df)):
     FP_df.loc[i, 'fpID'] = ((i+1)**2)+999
FP_df.head()

,yearID,teamID,playerID,salary,fpID
0,1985,ATL,barkele01,870000,1000
1,1985,ATL,bedrost01,550000,1003
2,1985,ATL,benedbr01,545000,1008
3,1985,ATL,campri01,633333,1015
4,1985,ATL,ceronri01,625000,1024


In [15]:
FP_df2 = pd.merge(FP_df, teamsDF, on=["teamID","yearID"])
FP_df3 = pd.DataFrame()
FP_df3['fpID'] = FP_df2['fpID']
FP_df3['franchiseID'] = FP_df2['franchiseID']
FP_df3['teamID'] = FP_df2['teamID'] 
FP_df3['yearID'] = FP_df2['yearID']
FP_df3['playerID'] = FP_df2['playerID']
FP_df3

,fpID,franchiseID,teamID,yearID,playerID
0,1000,ATL,ATL,1985,barkele01
1,1003,ATL,ATL,1985,bedrost01
2,1008,ATL,ATL,1985,benedbr01
3,1015,ATL,ATL,1985,campri01
4,1024,ATL,ATL,1985,ceronri01
...,...,...,...,...,...
26423,698228775,WSN,WAS,2016,strasst01
26424,698281624,WSN,WAS,2016,taylomi02
26425,698334475,WSN,WAS,2016,treinbl01
26426,698387328,WSN,WAS,2016,werthja01


In [16]:
FP_df3['teamIDy'] = FP_df3.index
for i in range(len(FP_df3)): 
     FP_df3.loc[i, 'teamIDy'] = FP_df3.loc[i,'teamID'] + "-" + str(FP_df3.loc[i,'yearID'])
FP_df3

,fpID,franchiseID,teamID,yearID,playerID,teamIDy
0,1000,ATL,ATL,1985,barkele01,ATL-1985
1,1003,ATL,ATL,1985,bedrost01,ATL-1985
2,1008,ATL,ATL,1985,benedbr01,ATL-1985
3,1015,ATL,ATL,1985,campri01,ATL-1985
4,1024,ATL,ATL,1985,ceronri01,ATL-1985
...,...,...,...,...,...,...
26423,698228775,WSN,WAS,2016,strasst01,WAS-2016
26424,698281624,WSN,WAS,2016,taylomi02,WAS-2016
26425,698334475,WSN,WAS,2016,treinbl01,WAS-2016
26426,698387328,WSN,WAS,2016,werthja01,WAS-2016


In [17]:
FP_df4 = FP_df3.copy()
FP_df4 = FP_df4[FP_df4.columns.drop('teamID')]
FP_df4 = FP_df4[FP_df4.columns.drop('yearID')]
FP_df4 = FP_df4.rename(columns={'teamIDy': 'teamID'})
FP_df4 = FP_df4.drop_duplicates()
FP_df4

,fpID,franchiseID,playerID,teamID
0,1000,ATL,barkele01,ATL-1985
1,1003,ATL,bedrost01,ATL-1985
2,1008,ATL,benedbr01,ATL-1985
3,1015,ATL,campri01,ATL-1985
4,1024,ATL,ceronri01,ATL-1985
...,...,...,...,...
26423,698228775,WSN,strasst01,WAS-2016
26424,698281624,WSN,taylomi02,WAS-2016
26425,698334475,WSN,treinbl01,WAS-2016
26426,698387328,WSN,werthja01,WAS-2016


In [18]:
salariesDF2 = FP_df.copy()
salariesDF2 = salariesDF2[salariesDF2.columns.drop('playerID')]
salariesDF2 = salariesDF2[salariesDF2.columns.drop('teamID')]
salariesDF2['salaryID'] = salariesDF2.index
salariesDF2.head()

,yearID,salary,fpID,salaryID
0,1985,870000,1000,0
1,1985,550000,1003,1
2,1985,545000,1008,2
3,1985,633333,1015,3
4,1985,625000,1024,4


In [19]:
for i in range(len(salariesDF2)):
     salariesDF2.loc[i, 'salaryID'] = ((i+1)**2)+111
salariesDF2

,yearID,salary,fpID,salaryID
0,1985,870000,1000,112
1,1985,550000,1003,115
2,1985,545000,1008,120
3,1985,633333,1015,127
4,1985,625000,1024,136
...,...,...,...,...
26423,2016,10400000,698228775,698227887
26424,2016,524000,698281624,698280736
26425,2016,524900,698334475,698333587
26426,2016,21733615,698387328,698386440


In [20]:
FP_df2 = FP_df.copy()
FP_df2 = FP_df2[FP_df2.columns.drop('yearID')]
FP_df2 = FP_df2[FP_df2.columns.drop('salary')]
FP_df2.head()

,teamID,playerID,fpID
0,ATL,barkele01,1000
1,ATL,bedrost01,1003
2,ATL,benedbr01,1008
3,ATL,campri01,1015
4,ATL,ceronri01,1024


In [21]:
# Read csv into a DataFrame
battingDF = pd.read_csv(pathBatting)
# Trim off unwanted columns
battingCols = ["playerID", "yearID","teamID", "stint", "G", "R", "H", "HR", "BB", "IBB", "SO", "SB"]
battingDF = battingDF[battingCols]
battingDF.head()

,playerID,yearID,teamID,stint,G,R,H,HR,BB,IBB,SO,SB
0,abercda01,1871,TRO,1,1,0,0,0,0,NaN,0.0,0.0
1,addybo01,1871,RC1,1,25,30,32,0,4,NaN,0.0,8.0
2,allisar01,1871,CL1,1,29,28,40,0,2,NaN,5.0,3.0
3,allisdo01,1871,WS3,1,27,28,44,2,0,NaN,2.0,1.0
4,ansonca01,1871,RC1,1,25,29,39,0,2,NaN,1.0,6.0


In [22]:
newBattingDF = pd.merge(battingDF, FP_df2, on="playerID")
newBattingDF = newBattingDF[newBattingDF.columns.drop('playerID')]
newBattingDF = newBattingDF[newBattingDF.columns.drop('teamID_x')]
newBattingDF = newBattingDF[newBattingDF.columns.drop('teamID_y')]
newBattingDF.head()

,yearID,stint,G,R,H,HR,BB,IBB,SO,SB,fpID
0,1954,1,35,2,10,0,3,NaN,15.0,0.0,336907024
1,1955,1,46,1,2,0,1,0.0,0.0,0.0,336907024
2,1956,1,49,4,11,0,2,0.0,28.0,0.0,336907024
3,1957,1,49,4,14,0,6,0.0,26.0,0.0,336907024
4,1958,1,46,1,6,0,5,0.0,13.0,1.0,336907024


In [23]:
newBattingDF[pd.isnull(newBattingDF).any(axis=1)]

,yearID,stint,G,R,H,HR,BB,IBB,SO,SB,fpID
0,1954,1,35,2,10,0,3,NaN,15.0,0.0,336907024


In [24]:
newBattingDF2 = newBattingDF.copy()
newBattingDF2['IBB'] = newBattingDF2['IBB'].fillna(0)
newBattingDF2[pd.isnull(newBattingDF2).any(axis=1)]

,yearID,stint,G,R,H,HR,BB,IBB,SO,SB,fpID


In [25]:
# Read csv into a DataFrame
pitchingDF = pd.read_csv(pathPitching)
# Trim off unwanted columns
pitchingCols = ["playerID", "yearID", "teamID","stint", "G", "H", "HR", "BB", "SO", "ERA", "R"]
pitchingDF = pitchingDF[pitchingCols]
pitchingDF.head()

,playerID,yearID,teamID,stint,G,H,HR,BB,SO,ERA,R
0,bechtge01,1871,PH1,1,3,43,0,11,1,7.96,42
1,brainas01,1871,WS3,1,30,361,4,37,13,4.50,292
2,fergubo01,1871,NY2,1,1,8,0,0,0,27.00,9
3,fishech01,1871,RC1,1,24,295,3,31,15,4.35,257
4,fleetfr01,1871,NY2,1,1,20,0,3,0,10.00,21


In [26]:
newPitchingDF = pd.merge(pitchingDF, FP_df2, on="playerID")
newPitchingDF = newPitchingDF[newPitchingDF.columns.drop('playerID')]
newPitchingDF = newPitchingDF[newPitchingDF.columns.drop('teamID_x')]
newPitchingDF = newPitchingDF[newPitchingDF.columns.drop('teamID_y')]
newPitchingDF.head()

,yearID,stint,G,H,HR,BB,SO,ERA,R,fpID
0,1954,1,35,141,17,72,72,3.74,71,336907024
1,1955,1,46,102,11,58,52,6.56,73,336907024
2,1956,1,49,210,26,71,96,3.99,109,336907024
3,1957,1,49,234,26,76,121,3.52,111,336907024
4,1958,1,46,194,12,55,74,4.13,89,336907024


In [27]:
newPitchingDF[pd.isnull(newPitchingDF).any(axis=1)]

,yearID,stint,G,H,HR,BB,SO,ERA,R,fpID
35492,1995,1,1,2,1,1,0,NaN,3,4025035
35493,1995,1,1,2,1,1,0,NaN,3,7129899
35494,1995,1,1,2,1,1,0,NaN,3,11581408
35495,1995,1,1,2,1,1,0,NaN,3,18089008
35496,1995,1,1,2,1,1,0,NaN,3,24523303
...,...,...,...,...,...,...,...,...,...,...
144110,2019,2,1,1,0,4,0,NaN,5,497781720
144111,2019,2,1,1,0,4,0,NaN,5,536479243
144112,2019,2,1,1,0,4,0,NaN,5,574657783
144113,2019,2,1,1,0,4,0,NaN,5,634385968


In [28]:
newPitchingDF2 = newPitchingDF.copy()
newPitchingDF2['ERA'] = newPitchingDF2['ERA'].fillna(0)
newPitchingDF2[pd.isnull(newPitchingDF2).any(axis=1)]

,yearID,stint,G,H,HR,BB,SO,ERA,R,fpID


<h1>===========================================================</h1>

<h1>Set up SQL DataBase</h1>

Before running of the cells below- <b>
    in pgAdmin: create a DataBase named 'baseballDB'

<h4>Create a connection to SQL database</h4>

In [29]:
pg_user = 'postgres'
pg_password = pgPassword
db_name = 'baseball_db'

connection_string = f"{pg_user}:{pg_password}@localhost:5432/{db_name}"
engine = create_engine(f'postgresql://{connection_string}')

In [30]:
engine.table_names()

['Franchises',
 'Team-Stats',
 'Teams',
 'FranchisePlayers',
 'Batting',
 'Pitching',
 'Salaries',
 'Tableau',
 'Players']

<h4>Add dataframes to SQL database</h4>

In [31]:
franchisesDF.to_sql(name = "Franchises", con = engine, if_exists='append', index=False)
print("franchise loaded success...")

franchise loaded success...


In [32]:
teamsDF2.to_sql(name = "Teams", con = engine, if_exists='append', index=False)
print("Teams loaded success...")

Teams loaded success...


In [33]:
team_statsDF2.to_sql(name = "Team-Stats", con = engine, if_exists='append', index=False)
print("Team-Stats loaded success...")

Team-Stats loaded success...


In [34]:
playersDF2.to_sql(name = "Players", con = engine, if_exists='append', index=False)
print("Players loaded success...")

Players loaded success...


In [35]:
FP_df4.to_sql(name = "FranchisePlayers", con = engine, if_exists='append', index=False)
print("FranchisePlayers loaded success...")

FranchisePlayers loaded success...


In [36]:
newBattingDF2.to_sql(name = "Batting", con = engine, if_exists='append', index=False)
print("Batting loaded success...")

Batting loaded success...


In [37]:
newPitchingDF2.to_sql(name = "Pitching", con = engine, if_exists='append', index=False)
print("Pitching loaded success...")

Pitching loaded success...


In [38]:
salariesDF2.to_sql(name = "Salaries", con = engine, if_exists='append', index=False)
print("Salaries loaded success...")

Salaries loaded success...
